In [1]:
import yfinance as yf
import pandas as pd
import datetime as dt
import requests as req
from bs4 import BeautifulSoup
import utils
from statistics import mean
import cash_flow
import formulas


In [2]:
tickers = ["TCS", "GNFC"]

In [3]:
ticker_data = []
years = []

for i in range(10):
        years.append(utils.get_financial_year() + i)

forward_fcf_df = pd.DataFrame(index=years)

for idx, ticker in enumerate(tickers):
    free_cash_flows = cash_flow.scraped_fcf(ticker)
    growth_rates = cash_flow.calculate_growth_rate(free_cash_flows)

    if mean(growth_rates) > 0:
        ticker_data.append({ ticker: { "avg_growth_rate": mean(growth_rates), "last_fcf_value": free_cash_flows[len(free_cash_flows) - 1] } })

        forward_fcf_df[ticker] = cash_flow.generate_forward_fcf(10, ticker_data, idx, ticker)

        ticker_info = utils.get_ticker_info(ticker ,ticker_data[idx][ticker])

        ticker_data[idx][ticker]["capm"] = formulas.CAPM(ticker_data[idx][ticker]["beta"])

        ticker_data[idx][ticker]["wacc"] = formulas.WACC(ticker_data[idx][ticker])


print(ticker_data)




TypeError: can only concatenate str (not "dict") to str

In [7]:
# PROJECT FUTURE FCF
t = yf.Ticker("TCS.NS")

print(t.balance_sheet)




Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []


In [10]:
# Get Cashflows
apv_api_key = '2N8UEPVRC2AWTK6B'

cashflows = req.get("https://www.alphavantage.co/query?function=CASH_FLOW&symbol=TCS&apikey=2N8UEPVRC2AWTK6B")
cashflows = cashflows.json()

print(cashflows)

# annual_cf_df = pd.DataFrame(cashflows['annualReports']).sort_index(ascending=False)
# dates = annual_cf_df['fiscalDateEnding'].values.tolist()
# years = []

# for date in dates:
#     years.append(dt.datetime.strptime(date, "%Y-%m-%d").year)

ing': '64394000', 'otherCashflowFromFinancing': '-373000', 'changeInOperatingActivities': '891000', 'netIncome': '14487000', 'changeInCash': '60391000', 'operatingCashflow': '30748000', 'otherOperatingCashflow': '1138000', 'depreciation': '38638000', 'dividendPayout': 'None', 'stockSaleAndPurchase': '-373000', 'changeInInventory': '-17293000', 'changeInAccountReceivables': '-1002000', 'changeInNetIncome': '3572000', 'capitalExpenditures': '33619000', 'changeInReceivables': '-1002000', 'changeInExchangeRate': '-1149000', 'changeInCashAndCashEquivalents': '60391000'}, {'fiscalDateEnding': '2019-03-31', 'reportedCurrency': 'USD', 'investments': '-32771000', 'changeInLiabilities': '13622000', 'cashflowFromInvestment': '-32771000', 'otherCashflowFromInvestment': '899000', 'netBorrowings': '-19495000', 'cashflowFromFinancing': '-22007000', 'otherCashflowFromFinancing': '-2384000', 'changeInOperatingActivities': '-3793000', 'netIncome': '21680000', 'changeInCash': '-1035000', 'operatingCashfl

In [ ]:
fcf_df = pd.DataFrame()
fcf_df["Year"] = years
fcf_df["Operating Cashflow"] = annual_cf_df["operatingCashflow"].astype(int)
fcf_df["Cap Ex"] = annual_cf_df["capitalExpenditures"].astype(int)
fcf_df["FCF"] = fcf_df["Operating Cashflow"] - fcf_df["Cap Ex"]
growth_rate = [0]

for i in range(len(fcf_df["FCF"])):
    if i < len(fcf_df["FCF"]) - 1:
        pct_diff = (fcf_df["FCF"].iloc[i+1] - fcf_df["FCF"].iloc[i])/abs(fcf_df["FCF"].iloc[i])
        growth_rate.append(pct_diff * 100)

fcf_df["Growth Rate"]  = growth_rate
avg_growth = fcf_df["Growth Rate"].iloc[1:].mean()

avg_growth